In [1]:
import polars as pl
import gc
import pickle
from pathlib import Path, PosixPath
from tqdm.auto import tqdm

import sys
sys.path.append('..')

from src.utils import seed_everything, get_logger, get_config, TimeUtil
from src.utils.competition_utils import clipping_input
from src.data import DataProvider, FeatureEngineering, Preprocessor, HFPreprocessor
from src.train import get_dataloader

In [2]:
# コマンドライン引数
exp = '146'

In [3]:
config = get_config(exp, config_dir=Path('../config'))
logger = get_logger(config.output_path)
logger.info(f'exp: {exp} | run_mode={config.run_mode}, multi_task={config.multi_task}, loss_type={config.loss_type}')

seed_everything(config.seed)

[ 2024-10-09 06:30:46 | INFO ] exp: 146 | run_mode=hf, multi_task=False, loss_type=mae


In [4]:
config.run_mode = 'debug'
config.multi_task = True

In [5]:
with TimeUtil.timer('Data Loading...'):
    dpr = DataProvider(config)
    train_df, test_df = dpr.load_data()

[Data Loading...] start [0.5GB(16.8%)]


[Data Loading...] done [56.7GB(20.3%)(+56.153GB)] 13.4642 s


In [6]:
with TimeUtil.timer('Feature Engineering...'):
    fer = FeatureEngineering(config)
    train_df = fer.feature_engineering(train_df)
    test_df = fer.feature_engineering(test_df)

[Feature Engineering...] start [56.7GB(20.1%)]


[Feature Engineering...] done [58.4GB(19.4%)(+1.793GB)] 7.4836 s


In [7]:
with TimeUtil.timer('Scaling and Clipping Features...'):
    ppr = Preprocessor(config)
    train_df, test_df = ppr.scaling(train_df, test_df)
    input_cols, target_cols = ppr.input_cols, ppr.target_cols
    if config.task_type == 'grid_pred':
        train_df = train_df.drop(target_cols)

    valid_df = train_df.filter(pl.col('fold') == 0)
    train_df = train_df.filter(pl.col('fold') != 0)
    valid_df, input_clip_dict = clipping_input(train_df, valid_df, input_cols)
    test_df, _ = clipping_input(None, test_df, input_cols, input_clip_dict)
    pickle.dump(input_clip_dict, open(config.output_path / 'input_clip_dict.pkl', 'wb'))

[Scaling and Clipping Features...] start [58.4GB(19.4%)]


[Scaling and Clipping Features...] done [58.3GB(18.0%)(-0.135GB)] 3.8249 s


In [8]:
with TimeUtil.timer('Converting to arrays for NN...'):
    array_data = ppr.convert_numpy_array(train_df, valid_df, test_df)
    del train_df, valid_df, test_df
    gc.collect()

[Converting to arrays for NN...] start [58.3GB(17.6%)]
[Converting to arrays for NN...] done [69.3GB(21.7%)(+10.971GB)] 46.5510 s


In [9]:
# Prepare HF Data
if config.run_mode == 'hf':
    with TimeUtil.timer('HF Data Preprocessing...'):
        hf_ppr = HFPreprocessor(config)
        # hf_pcr.preprocess_data()
        # hf_pcr.convert_numpy_array(near_target=False)
        # del train_loader; gc.collect()
        # train_loader = None

In [10]:
with TimeUtil.timer('Creating Torch DataLoader...'):
    train_loader = get_dataloader(
        config,
        array_data['train_ids'],
        array_data['X_train'],
        array_data['y_train'],
        is_train=True
    )
    valid_loader = get_dataloader(
        config,
        array_data['valid_ids'],
        array_data['X_valid'],
        array_data['y_valid'],
        is_train=False
    )
    test_loader = get_dataloader(
        config,
        array_data['test_ids'],
        array_data['X_test'],
        is_train=False
    )
    del array_data
    gc.collect()

[Creating Torch DataLoader...] start [69.3GB(21.4%)]
[Creating Torch DataLoader...] done [69.3GB(21.4%)(+0.000GB)] 0.1442 s


# Model

In [11]:
import torch
import torch.nn.functional as F
from torch import nn

from src.utils.constant import VERTICAL_INPUT_COLS, ADDITIONAL_VERTICAL_INPUT_COLS

class BaseModel(nn.Module):
    """
    Base class for all LEAP models
    """
    def __init__(self):
        super().__init__()
        self.num_vertical = len(VERTICAL_INPUT_COLS) + len(ADDITIONAL_VERTICAL_INPUT_COLS)

    def forward(self, x):
        v_x = x[:, :, :self.num_vertical]
        diff_feat = self.calc_diff_feats(v_x)
        x = torch.cat([x, diff_feat], dim=-1)
        return x

    def calc_lag_features(self, x):
        feat = []
        # Calculate lag features for both forward and backward directions
        for t in [1, 2, 3, 4, 5]:
            x_f = torch.cat(
                [torch.zeros(x.size(0), t, x.size(2), device=x.device), x[:, : (60 - t), :]], dim=1
            )
            feat.append(x_f)
            x_b = torch.cat(
                [x[:, t:, :], torch.zeros(x.size(0), t, x.size(2), device=x.device)], dim=1
            )
            feat.append(x_b)
        feat = torch.cat(feat, dim=-1)
        return feat

    def calc_diff_feats(self, x):
        # Forward direction diff
        forward_diff = torch.diff(x, dim=1)
        forward_diff = torch.cat([torch.zeros_like(x[:, 0, :]).unsqueeze(1), forward_diff], dim=1)
        # Forward direction second diff
        forward_diff2 = torch.diff(forward_diff, dim=1)
        forward_diff2 = torch.cat([torch.zeros_like(x[:, 0, :]).unsqueeze(1), forward_diff2], dim=1)
        # Backward direction diff
        backward_diff = torch.diff(x.flip(1), dim=1).flip(1)
        backward_diff = torch.cat([backward_diff, torch.zeros_like(x[:, 0, :]).unsqueeze(1)], dim=1)
        # Backward direction second diff
        backward_diff2 = torch.diff(backward_diff.flip(1), dim=1).flip(1)
        backward_diff2 = torch.cat(
            [backward_diff2, torch.zeros_like(x[:, 0, :]).unsqueeze(1)], dim=1
        )
        feat = torch.cat([forward_diff, forward_diff2, backward_diff, backward_diff2], dim=-1)
        return feat

    def calc_moving_feats(self, x):
        feat = []
        x = x.transpose(1, 2)  # (batch, hidden, sequence)
        # Moving statistics
        for w in [3, 5, 7, 15, 29]:
            # Mean, max, min
            feat.append(F.avg_pool1d(x, w, stride=1, padding=w // 2))
            feat.append(F.max_pool1d(x, w, stride=1, padding=w // 2))
            feat.append(-1 * F.max_pool1d(-1 * x, w, stride=1, padding=w // 2))
            # Standard deviation
            x_mean = F.avg_pool1d(x, w, stride=1, padding=w // 2)
            x_diff = (x - x_mean) ** 2
            x_std = F.avg_pool1d(x_diff, w, stride=1, padding=w // 2).sqrt()
            x_std = torch.where(torch.isinf(x_std) | torch.isnan(x_std), 0, x_std)
            feat.append(x_std)

        # Global statistics
        feat.append(x.mean(dim=2, keepdim=True).repeat(1, 1, x.size(2)))
        feat.append(x.max(dim=2, keepdim=True).values.repeat(1, 1, x.size(2)))
        feat.append(x.min(dim=2, keepdim=True).values.repeat(1, 1, x.size(2)))
        x_std = x.std(dim=2, keepdim=True).repeat(1, 1, x.size(2))
        x_std = torch.where(torch.isinf(x_std) | torch.isnan(x_std), 0, x_std)
        feat.append(x_std)

        feat = torch.cat(feat, dim=1)
        feat = feat.transpose(1, 2)
        return feat


In [57]:
from src.model.modules import VerticalEncoding, ResNetBlock, ConvExtractor, LSTMBlock

In [48]:
for x, y in valid_loader:
    break

In [49]:
embedding = nn.Sequential(
    nn.Conv1d(30, config.hidden_dim, kernel_size=1, stride=1, padding=0),
    nn.BatchNorm1d(config.hidden_dim),
    nn.ELU(),
)
x = x.transpose(1, 2)
x = embedding(x)
x = x.transpose(1, 2)

In [50]:
x.shape

torch.Size([4096, 60, 256])

In [51]:
g_id = torch.randint(0, 384, (config.eval_batch, ))
he = VHPositionalEncoding(config.hidden_dim)
out = he(x, g_id)

In [47]:
out.size()

torch.Size([4096, 60, 256])

In [ ]:
class Conv1DSeq2Seq(BaseModel):
    def __init__(
        self,
        in_dim: int,
        out_dim: int,
        hidden_dim: int = 256,
        block_num: int = 15,
        kernel_size: int = 5,
        multitask: bool = False,
    ):
        super().__init__()
        if multitask:
            out_dim = 3 * out_dim

        activation = nn.ELU()
        self.embedding = nn.Sequential(
            nn.Conv1d(in_dim, hidden_dim, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm1d(hidden_dim),
            activation,
        )
        self.pe = VerticalEncoding(hidden_dim, learnable=True)
        conv_blocks = []
        for i in range(block_num):
            if i >= (block_num // 2) and i < (block_num - 1):
                conv_blocks.append(
                    ResNetBlock(hidden_dim, hidden_dim, activation=activation, kernel_size=kernel_size, inception=False)
                )
            else:
                conv_blocks.append(
                    ResNetBlock(hidden_dim, hidden_dim, activation=activation, kernel_size=kernel_size, inception=True)
                )
        self.conv_blocks = nn.ModuleList(conv_blocks)
        self.head = nn.Sequential(
            nn.Conv1d(hidden_dim, 64, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, out_dim, kernel_size=1, stride=1, padding=0),
        )


    def forward(self, x):
        x = super().forward(x)
        x = x.transpose(1, 2)
        x = self.embedding(x)
        x = x.transpose(1, 2)
        x = self.pe(x)
        x = x.transpose(1, 2)
        for conv_block in self.conv_blocks:
            x = conv_block(x)
        out = self.head(x)
        return out.transpose(1, 2)

In [ ]:
from src.torch.models.modules import (
    SequentialStats,
    VerticalEncoding,
    ResNetBlock,
    ConvExtractor,
    LSTMBlock,
)

class Conv1DSeq2Seq(nn.Module):
    def __init__(
        self,
        in_dim: int,
        out_dim: int,
        hidden_dim: int = 256,
        block_num: int = 15,
        kernel_size: int = 5,
    ):
        super().__init__()
        activation = nn.ELU()
        self.seq_stats = SequentialStats()
        self.embedding = nn.Sequential(
            nn.Conv1d(in_dim, hidden_dim, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm1d(hidden_dim),
            activation,
        )
        self.pe = VerticalEncoding(hidden_dim, learnable=True)
        conv_blocks = []
        for i in range(block_num):
            if i >= (block_num // 2) and i < (block_num - 1):
                conv_blocks.append(
                    ResNetBlock(hidden_dim, hidden_dim, activation=activation, kernel_size=kernel_size, inception=False)
                )
            else:
                conv_blocks.append(
                    ResNetBlock(hidden_dim, hidden_dim, activation=activation, kernel_size=kernel_size, inception=True)
                )
        self.conv_blocks = nn.ModuleList(conv_blocks)
        self.head = nn.Sequential(
            nn.Conv1d(hidden_dim, 64, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, out_dim, kernel_size=1, stride=1, padding=0),
        )

    def forward(self, x):
        x = self.seq_stats(x)
        x = x.transpose(1, 2)
        x = self.embedding(x)
        x = x.transpose(1, 2)
        x = self.pe(x)
        x = x.transpose(1, 2)
        for conv_block in self.conv_blocks:
            x = conv_block(x)
        out = self.head(x)
        return out.transpose(1, 2)

# END

In [ ]:
import loguru
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from omegaconf import OmegaConf, DictConfig

from src.utils.constant import VERTICAL_TARGET_COLS
from src.train import ComponentFactory
from src.train.train_utils import AverageMeter


class Trainer:
    def __init__(self, config: DictConfig, logger: loguru._Logger):
        self.config = config
        self.logger = logger

        self.factory = ComponentFactory(config)
        self.model = self.factory.get_model()
        self.model = self.model.to(config.device)
        n_device = torch.cuda.device_count()
        if n_device > 1:
            self.model = nn.DataParallel(self.model)
        self.loss_fn = self.factory.get_loss()

        self.train_loss = AverageMeter()
        self.valid_loss = AverageMeter()

        self.y_numerators = pickle.load(open(config.output_path / f'y_numerators_{config.target_scale_method}.npy', 'rb'))
        self.y_denominators = pickle.load(open(config.output_path / f'y_denominators_{config.target_scale_method}.npy', 'rb'))


    def train(self, train_loader: DataLoader, valid_loader: DataLoader, detail_pbar: bool = True):

        self.optimizer = self.factory.get_optimizer()
        self.scheduler = self.factory.get_scheduler(steps_per_epoch=len(train_loader))

        global_step = 0

        for epoch in tqdm(range(self.config.epochs)):
            self.model.train()
            self.train_loss.reset()

            iterations = tqdm(train_loader, total=len(train_loader)) if self.detail_pbar else train_loader
            for data in iterations:
                _, loss = self.forward_step(data, calc_loss=True)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                self.train_loss.update(loss.item(), n=data[0].size(0))
                global_step += 1

                if global_step % self.config.eval_step == 0:
                    score, _, preds = self.valid_evaluate(valid_loader, epoch, eval_count)
                    # if colwise_best_weight:
                    #     torch.save(self.model.state_dict(), self.output_path / f'model{self.save_suffix}_eval{eval_count}.pth')
                    # if score > best_score:
                    #     best_score = score
                    #     best_preds = preds
                    #     best_epochs = epoch
                    #     torch.save(self.model.state_dict(), self.output_path / f'model{self.save_suffix}_best.pth')
                    # eval_count += 1
                    # self.model.train()

    def valid_evaluate(
            self,
            valid_loader: DataLoader,
            current_epoch: int,
            eval_count: int,
            eval_colwise: bool = False,
            load_best_weight: bool = False,
        ):
            if self.valid_ids is None:
                self.valid_ids = valid_loader.dataset.sample_ids

            if eval_colwise:
                preds = self.inference_loop_colwise(valid_loader, 'valid', self.best_score_dict)
            else:
                preds = self.inference_loop(valid_loader, 'valid', load_best_weight)

            labels = valid_loader.dataset.y
            if self.config.target_shape == '3dim':
                labels = self.convert_target_3dim_to_2dim(labels)
            labels = self.restore_pred(labels)

            if self.pp_run and self.valid_pp_x is None:
                self.load_input_for_postprocess('valid')
            if self.pp_run:
                preds = self.postprocess(preds, run_type='valid')
            if self.out_clip:
                preds = self.clipping_pred(preds)

            eval_idx = [i for i, col in enumerate(self.target_cols) if self.factor_dict[col] != 0]  # factor_dictの値が0のものは自動でR2=1になるようにする
            score, indiv_score = evaluate_metric(preds, labels, individual=True, eval_idx=eval_idx)
            save_dict = False if load_best_weight else True # 通常の学習ループの時のみbest_score_dictの保存を行う
            colwise_score = self.update_best_score(indiv_score, eval_count, save_dict=save_dict)
            message = f"""
                [Valid] :
                    Epoch={current_epoch},
                    Loss={self.valid_loss.avg:.7f},
                    Score={score:.5f},
                    Best Col-Wise Score={colwise_score:.5f}
            """
            self.logger.info(clean_message(message))
            return score, colwise_score, preds

    def forward_step(self, data: torch.Tensor, calc_loss: bool = True):
        if calc_loss:
            x, y = data
            x, y = x.to(self.device), y.to(self.device)
            out = self.model(x)
            if self.config.target_shape == '3dim':
                out = self.convert_target_3dim_to_2dim(out)
                y = self.convert_target_3dim_to_2dim(y)
            loss = self.loss_fn(out, y)
            return out, loss
        else:
            x = data
            x = x.to(self.device)
            out = self.model(x)
            if self.config.target_shape == '3dim':
                out = self.convert_target_3dim_to_2dim(out)
            return out, None

    def inference_loop(
        self,
        test_loader: DataLoader,
        mode: Literal['valid', 'test'],
        load_best_weight: bool = False
    ):
        self.model.eval()
        if mode == 'valid':
            self.valid_loss.reset()

        if load_best_weight:
            self.model.load_state_dict(torch.load(self.output_path / f'model{self.save_suffix}_best.pth'))

        preds = []
        with torch.no_grad():
            iterations = tqdm(test_loader, total=len(test_loader)) if self.detail_pbar else test_loader
            for batched in iterations:
                if mode == 'valid':
                    out, loss = self.forward_step(batched, calc_loss=True)
                    self.valid_loss.update(loss.item(), n=batched[0].size(0))
                elif mode == 'test':
                    out, _ = self.forward_step(batched, calc_loss=False)
                preds.append(out.detach().cpu().numpy())
        preds = np.concatenate(preds, axis=0)
        preds = self.restore_pred(preds)
        return preds


    def alignment_target_idx(self, y: np.ndarray | torch.Tensor) -> np.ndarray | torch.Tensor:
        if self.seq_target_cols is None:
            seq_target_cols = []
            for col in VERTICAL_TARGET_COLS:
                seq_target_cols.extend([f'{col}_{i}' for i in range(60)])
            for col in SCALER_TARGET_COLS:
                seq_target_cols.append(col)
            self.seq_target_cols = seq_target_cols
        align_order = [self.seq_target_cols.index(col) for col in self.target_cols]
        assert len(y.shape) == 2
        y = y[:, align_order]
        return y

    def convert_target_3dim_to_2dim(self, y: np.ndarray | torch.Tensor) -> np.ndarray | torch.Tensor:
        y_v = y[:, :, :len(VERTICAL_TARGET_COLS)]
        y_s = y[:, :, len(VERTICAL_TARGET_COLS):]
        if type(y) == np.ndarray:
        # if isinstance(y, np.ndarray):
            y_v = np.transpose(y_v, (0, 2, 1)).reshape(y.shape[0], -1)
            y_s = y_s.mean(axis=1)
            y = np.concatenate([y_v, y_s], axis=-1)
        else:
            y_v = y_v.permute(0, 2, 1).reshape(y.size(0), -1)
            y_s = y_s.mean(dim=1)
            y = torch.cat([y_v, y_s], dim=-1)
        y = self.alignment_target_idx(y)
        return y

    def restore_pred(self, preds: np.ndarray):
        return preds * self.config.y_denominators + self.config.y_numerators

        ########################################
        if eval_only:
            self.best_score_dict = pickle.load(open(self.output_path / f'best_score_dict{self.save_suffix}.pkl', 'rb'))
            score, _, preds = self.valid_evaluate(valid_loader, -1, -1, eval_colwise, load_best_weight=True)
            self.save_oof_df(preds, self.valid_ids, self.target_cols)
            return score, -1
        if retrain:
            self.best_score_dict = pickle.load(open(self.output_path / f'best_score_dict{self.save_suffix}.pkl', 'rb'))
            self.model.load_state_dict(torch.load(self.output_path / f'{retrain_weight_name}.pth'))

        global_step = 0
        eval_count = 0 if retrain_eval_count is None else retrain_eval_count + 1
        best_score = -np.inf if retrain_best_score is None else retrain_best_score
        step_per_epoch = len(train_loader) if self.run_mode != 'hf' else self.config.eval_step
        self.init_optimizer_and_scheduler(step_per_epoch=step_per_epoch)
        for epoch in tqdm(range(self.config.epochs)):
            self.model.train()
            self.train_loss.reset()
            if self.run_mode == 'hf':
                del train_loader; gc.collect()
                train_loader = self.get_train_loader_from_hf()

            iterations = tqdm(train_loader, total=len(train_loader)) if self.detail_pbar else train_loader
            for batched in iterations:
                _, loss = self.forward_step(batched, calc_loss=True)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                self.train_loss.update(loss.item(), n=batched[0].size(0))
                global_step += 1

                if global_step % self.config.eval_step == 0:
                    score, _, preds = self.valid_evaluate(valid_loader, epoch, eval_count)
                    if colwise_best_weight:
                        torch.save(self.model.state_dict(), self.output_path / f'model{self.save_suffix}_eval{eval_count}.pth')
                    if score > best_score:
                        best_score = score
                        best_preds = preds
                        best_epochs = epoch
                        torch.save(self.model.state_dict(), self.output_path / f'model{self.save_suffix}_best.pth')
                    eval_count += 1
                    self.model.train()

            message = f"""
                [Train] :
                    Epoch={epoch},
                    Loss={self.train_loss.avg:.7f},
                    LR={self.optimizer.param_groups[0]["lr"]:.4e}
            """
            self.logger.info(clean_message(message))

        pickle.dump(self.best_score_dict, open(self.output_path / f'best_score_dict{self.save_suffix}.pkl', 'wb'))
        if colwise_best_weight:
            best_score, _, best_preds = self.valid_evaluate(valid_loader, -1, -1, eval_colwise=True, load_best_weight=True)
            self.remove_unuse_weights()

        self.save_oof_df(best_preds, self.valid_ids, self.target_cols)
        return best_score, best_epochs # 全体スコアが最高の時のEpoch

In [ ]:
class Trainer:
    def __init__(
        self,
        config,
        model: nn.Module,
        loss_fn: nn.Module | None = None,
        save_suffix: str = '',
        logger: loguru._Logger | None = None,
        detail_pbar: bool = True
    ):
        self.config = config
        self.run_mode = config.run_mode
        self.device = config.device
        self.input_path = config.input_path
        self.output_path = config.output_path
        self.oof_path = config.oof_path

        self.target_cols = config.target_cols
        self.mul_old_factor = config.mul_old_factor
        self.target_min_max = [TARGET_MIN_MAX[col] for col in config.target_cols]
        self.factor_dict = get_sub_factor(config.input_path, old=False)
        self.old_factor_dict = get_sub_factor(config.input_path, old=True)
        self.out_clip = config.out_clip

        self.model = model
        self.model.to(self.device)
        self.loss_fn = loss_fn
        self.train_loss = AverageMeter()
        self.valid_loss = AverageMeter()
        self.save_suffix = save_suffix
        self.logger = logger
        self.detail_pbar = detail_pbar

        self.hf_ym_list = []
        self.valid_ids = None
        self.test_ids = None
        self.seq_target_cols = None
        self.pp_x_cols = [f'state_q0002_{i}' for i in range(12, 27)]
        self.pp_y_cols = [f'ptend_q0002_{i}' for i in range(12, 27)]
        self.pp_run = len(set(self.target_cols) & set(self.pp_y_cols)) > 0
        self.valid_pp_x = None
        self.test_pp_x = None
        self.best_score_dict = {}

    def init_optimizer_and_scheduler(self, step_per_epoch: int):
        optimizer = get_optimizer(
            self.model,
            method=self.config.optimizer_method,
            lr=self.config.lr,
            weight_decay=self.config.weight_decay,
            betas=self.config.betas
        )
        if self.config.scheduler_method == 'linear':
            total_steps = self.config.epochs * step_per_epoch
            scheduler_args = {
                'start_factor': self.config.linear_start_factor,
                'end_factor': self.config.linear_end_factor,
                'total_iters': total_steps * self.config.linear_end_step_ratio,
            }
        elif self.config.scheduler_method == 'multistep':
            scheduler_args = {
                'milestones': self.config.multi_milestones,
                'gamma': self.config.multi_gamma,
            }
        elif self.config.scheduler_method == 'cosine':
            T_0 = self.config.cosine_t0_epoch * step_per_epoch
            scheduler_args = {
                'T_0': T_0,
                'T_mult': self.config.cosine_t_mult,
                'eta_min': self.config.cosine_min_lr,
                'warmup_steps': self.config.cosine_warmup_steps,
                'gamma': self.config.cosine_gamma
            }
        scheduler = get_scheduler(
            optimizer,
            method=self.config.scheduler_method,
            scheduler_args=scheduler_args
        )
        self.optimizer = optimizer
        self.scheduler = scheduler

    def get_train_loader_from_hf(self, files_per_epoch: int = 5):
        npy_path = self.config.input_path / 'huggingface' / 'npy'
        if len(self.hf_ym_list) == 0:
            npy_files = list(npy_path.glob('X_*.npy'))
            self.hf_ym_list = [file.stem.split('_')[1] for file in npy_files]
        ym_extract = random.sample(self.hf_ym_list, min(files_per_epoch, len(self.hf_ym_list)))
        self.hf_ym_list = [ym for ym in self.hf_ym_list if ym not in ym_extract]
        X, y = [], []
        for ym in ym_extract:
            X.append(np.load(npy_path / f'X_{ym}.npy'))
            y.append(np.load(npy_path / f'y_{ym}.npy'))
        X = np.concatenate(X, axis=0)
        y = np.concatenate(y, axis=0)
        train_loader = get_dataloader(self.config, sample_ids=None, X=X, y=y, is_train=True)
        del X, y; gc.collect()
        return train_loader

    def train(
        self,
        train_loader: DataLoader | None,
        valid_loader: DataLoader,
        colwise_best_weight: bool = False,
        eval_only: bool = False,
        eval_colwise: bool = False,
        retrain: bool = False,
        retrain_weight_name: str | None = None,
        retrain_best_score: float | None = None,
        retrain_eval_count: int | None = None, # 前回の最終eval_countを指定する
    ) -> Tuple[float, int]:

        if eval_only:
            self.best_score_dict = pickle.load(open(self.output_path / f'best_score_dict{self.save_suffix}.pkl', 'rb'))
            score, _, preds = self.valid_evaluate(valid_loader, -1, -1, eval_colwise, load_best_weight=True)
            self.save_oof_df(preds, self.valid_ids, self.target_cols)
            return score, -1
        if retrain:
            self.best_score_dict = pickle.load(open(self.output_path / f'best_score_dict{self.save_suffix}.pkl', 'rb'))
            self.model.load_state_dict(torch.load(self.output_path / f'{retrain_weight_name}.pth'))

        global_step = 0
        eval_count = 0 if retrain_eval_count is None else retrain_eval_count + 1
        best_score = -np.inf if retrain_best_score is None else retrain_best_score
        step_per_epoch = len(train_loader) if self.run_mode != 'hf' else self.config.eval_step
        self.init_optimizer_and_scheduler(step_per_epoch=step_per_epoch)
        for epoch in tqdm(range(self.config.epochs)):
            self.model.train()
            self.train_loss.reset()
            if self.run_mode == 'hf':
                del train_loader; gc.collect()
                train_loader = self.get_train_loader_from_hf()

            iterations = tqdm(train_loader, total=len(train_loader)) if self.detail_pbar else train_loader
            for batched in iterations:
                _, loss = self.forward_step(batched, calc_loss=True)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                self.train_loss.update(loss.item(), n=batched[0].size(0))
                global_step += 1

                if global_step % self.config.eval_step == 0:
                    score, _, preds = self.valid_evaluate(valid_loader, epoch, eval_count)
                    if colwise_best_weight:
                        torch.save(self.model.state_dict(), self.output_path / f'model{self.save_suffix}_eval{eval_count}.pth')
                    if score > best_score:
                        best_score = score
                        best_preds = preds
                        best_epochs = epoch
                        torch.save(self.model.state_dict(), self.output_path / f'model{self.save_suffix}_best.pth')
                    eval_count += 1
                    self.model.train()

            message = f"""
                [Train] :
                    Epoch={epoch},
                    Loss={self.train_loss.avg:.7f},
                    LR={self.optimizer.param_groups[0]["lr"]:.4e}
            """
            self.logger.info(clean_message(message))

        pickle.dump(self.best_score_dict, open(self.output_path / f'best_score_dict{self.save_suffix}.pkl', 'wb'))
        if colwise_best_weight:
            best_score, _, best_preds = self.valid_evaluate(valid_loader, -1, -1, eval_colwise=True, load_best_weight=True)
            self.remove_unuse_weights()

        self.save_oof_df(best_preds, self.valid_ids, self.target_cols)
        return best_score, best_epochs # 全体スコアが最高の時のEpoch

    def valid_evaluate(
        self,
        valid_loader: DataLoader,
        current_epoch: int,
        eval_count: int,
        eval_colwise: bool = False,
        load_best_weight: bool = False,
    ):
        if self.valid_ids is None:
            self.valid_ids = valid_loader.dataset.sample_ids

        if eval_colwise:
            preds = self.inference_loop_colwise(valid_loader, 'valid', self.best_score_dict)
        else:
            preds = self.inference_loop(valid_loader, 'valid', load_best_weight)

        labels = valid_loader.dataset.y
        if self.config.target_shape == '3dim':
            labels = self.convert_target_3dim_to_2dim(labels)
        labels = self.restore_pred(labels)

        if self.pp_run and self.valid_pp_x is None:
            self.load_input_for_postprocess('valid')
        if self.pp_run:
            preds = self.postprocess(preds, run_type='valid')
        if self.out_clip:
            preds = self.clipping_pred(preds)

        eval_idx = [i for i, col in enumerate(self.target_cols) if self.factor_dict[col] != 0]  # factor_dictの値が0のものは自動でR2=1になるようにする
        score, indiv_score = evaluate_metric(preds, labels, individual=True, eval_idx=eval_idx)
        save_dict = False if load_best_weight else True # 通常の学習ループの時のみbest_score_dictの保存を行う
        colwise_score = self.update_best_score(indiv_score, eval_count, save_dict=save_dict)
        message = f"""
            [Valid] :
                Epoch={current_epoch},
                Loss={self.valid_loss.avg:.7f},
                Score={score:.5f},
                Best Col-Wise Score={colwise_score:.5f}
        """
        self.logger.info(clean_message(message))
        return score, colwise_score, preds

    def test_predict(self, test_loader: DataLoader, eval_colwise: bool = False) -> pl.DataFrame:
        if self.test_ids is None:
            self.test_ids = test_loader.dataset.sample_ids

        if eval_colwise:
            self.best_score_dict = pickle.load(open(self.output_path / f'best_score_dict{self.save_suffix}.pkl', 'rb'))
            preds = self.inference_loop_colwise(test_loader, 'test', self.best_score_dict)
        else:
            preds = self.inference_loop(test_loader, 'test', load_best_weight=True)

        if self.pp_run and self.test_pp_x is None:
            self.load_input_for_postprocess('test')
        if self.pp_run:
            preds = self.postprocess(preds, run_type='test')
        if self.out_clip:
            preds = self.clipping_pred(preds)

        pred_df = pl.DataFrame(preds, schema=self.target_cols)
        pred_df = pred_df.with_columns(sample_id = pl.Series(self.test_ids))
        return pred_df

    def forward_step(self, batched: torch.Tensor, calc_loss: bool = True):
        if calc_loss:
            x, y = batched
            x, y = x.to(self.device), y.to(self.device)
            out = self.model(x)
            if self.config.target_shape == '3dim':
                out = self.convert_target_3dim_to_2dim(out)
                y = self.convert_target_3dim_to_2dim(y)
            loss = self.loss_fn(out, y)
            return out, loss
        else:
            x = batched
            x = x.to(self.device)
            out = self.model(x)
            if self.config.target_shape == '3dim':
                out = self.convert_target_3dim_to_2dim(out)
            return out, None

    def inference_loop(
        self,
        test_loader: DataLoader,
        mode: Literal['valid', 'test'],
        load_best_weight: bool = False
    ):
        self.model.eval()
        if mode == 'valid':
            self.valid_loss.reset()
        if load_best_weight:
            self.model.load_state_dict(torch.load(self.output_path / f'model{self.save_suffix}_best.pth'))

        preds = []
        with torch.no_grad():
            iterations = tqdm(test_loader, total=len(test_loader)) if self.detail_pbar else test_loader
            for batched in iterations:
                if mode == 'valid':
                    out, loss = self.forward_step(batched, calc_loss=True)
                    self.valid_loss.update(loss.item(), n=batched[0].size(0))
                elif mode == 'test':
                    out, _ = self.forward_step(batched, calc_loss=False)
                preds.append(out.detach().cpu().numpy())
        preds = np.concatenate(preds, axis=0)
        preds = self.restore_pred(preds)
        return preds

    def inference_loop_colwise(
        self,
        test_loader: DataLoader,
        mode: Literal['valid', 'test'],
        best_score_dict: Dict[str, Tuple[int, float]],
    ):
        self.model.eval()
        if mode == 'valid':
            self.valid_loss.reset()

        use_evals = list(set([eval_count for _, (eval_count, _) in best_score_dict.items()]))
        final_preds = np.zeros((len(test_loader.dataset), len(self.target_cols)))
        for eval_count in tqdm(use_evals, desc='Inference Col-Wise Weight'):
            self.model.load_state_dict(torch.load(self.output_path / f'model{self.save_suffix}_eval{eval_count}.pth'))
            preds = []
            with torch.no_grad():
                iterations = tqdm(test_loader, total=len(test_loader)) if self.detail_pbar else test_loader
                for batched in iterations:
                    if mode == 'valid':
                        out, loss = self.forward_step(batched, calc_loss=True)
                        self.valid_loss.update(loss.item(), n=batched[0].size(0))
                    elif mode == 'test':
                        out, _ = self.forward_step(batched, calc_loss=False)
                    preds.append(out.detach().cpu().numpy())
            preds = np.concatenate(preds, axis=0)
            preds = self.restore_pred(preds)

            target_cols = [col for col, (count, _) in best_score_dict.items() if count == eval_count]
            for col in target_cols:
                idx = self.target_cols.index(col)
                final_preds[:, idx] = preds[:, idx]
        return final_preds

    def update_best_score(self, indiv_score: List[float], eval_count: int, save_dict: bool):
        for col, score in zip(self.target_cols, indiv_score):
            if col not in self.best_score_dict or score > self.best_score_dict[col][1]:
                self.best_score_dict[col] = (eval_count, score)
        best_colwise_score = (np.sum([score for _, score in self.best_score_dict.values()]) + (368 - len(self.target_cols))) / 368
        if save_dict:
            pickle.dump(self.best_score_dict, open(self.output_path / f'best_score_dict{self.save_suffix}.pkl', 'wb'))
        return best_colwise_score

    def remove_unuse_weights(self):
        use_eval_counts = set([v[0] for v in self.best_score_dict.values()])
        weight_paths = list(self.output_path.glob(f'model{self.save_suffix}_eval*.pth'))
        for path in weight_paths:
            eval_count = int(path.stem.split('_')[-1].replace('eval', ''))
            if eval_count not in use_eval_counts:
                path.unlink()

    def convert_target_3dim_to_2dim(self, y: np.ndarray | torch.Tensor) -> np.ndarray | torch.Tensor:
        y_v = y[:, :, :len(VERTICAL_TARGET_COLS)]
        y_s = y[:, :, len(VERTICAL_TARGET_COLS):]
        if type(y) == np.ndarray:
            y_v = np.transpose(y_v, (0, 2, 1)).reshape(y.shape[0], -1)
            y_s = y_s.mean(axis=1)
            y = np.concatenate([y_v, y_s], axis=-1)
        else:
            y_v = y_v.permute(0, 2, 1).reshape(y.size(0), -1)
            y_s = y_s.mean(dim=1)
            y = torch.cat([y_v, y_s], dim=-1)
        y = self.alignment_target_idx(y)
        return y

    def alignment_target_idx(self, y: np.ndarray | torch.Tensor) -> np.ndarray | torch.Tensor:
        if self.seq_target_cols is None:
            seq_target_cols = []
            for col in VERTICAL_TARGET_COLS:
                seq_target_cols.extend([f'{col}_{i}' for i in range(60)])
            for col in SCALER_TARGET_COLS:
                seq_target_cols.append(col)
            self.seq_target_cols = seq_target_cols
        align_order = [self.seq_target_cols.index(col) for col in self.target_cols]
        assert len(y.shape) == 2
        y = y[:, align_order]
        return y

    def restore_pred(self, preds: np.ndarray):
        return preds * self.config.y_denominators + self.config.y_numerators

    def clipping_pred(self, preds: np.ndarray):
        for i in range(preds.shape[1]):
            preds[:, i] = np.clip(
                preds[:, i],
                self.target_min_max[i][0],
                self.target_min_max[i][1]
            )
        return preds

    def postprocess(self, preds: np.ndarray, run_type: Literal['valid', 'test']):
        pp_x = self.valid_pp_x if run_type == 'valid' else self.test_pp_x
        for y_col, x_col in zip(self.pp_y_cols, self.pp_x_cols):
            if y_col in self.target_cols:
                idx = self.target_cols.index(y_col)
                old_factor = self.old_factor_dict[y_col] if self.mul_old_factor else 1
                preds[:, idx] = (-1 * pp_x[x_col].to_numpy() / 1200) * old_factor
        return preds

    def load_input_for_postprocess(self, data_type: Literal['valid', 'test']):
        if data_type == 'valid':
            self.valid_pp_x = (
                pl.scan_parquet(self.input_path / 'train_pp.parquet')
                .select(['sample_id'] + self.pp_x_cols)
                .filter(pl.col('sample_id').is_in(self.valid_ids))
                .collect()
            )
            id_df = pl.DataFrame({'sample_id': self.valid_ids})
            self.valid_pp_x = id_df.join(self.valid_pp_x, on='sample_id', how='left')

        elif data_type == 'test':
            self.test_pp_x = pl.read_parquet(
                Config.input_path / 'test_pp.parquet',
                columns=['sample_id'] + self.pp_x_cols
            )
            id_df = pl.DataFrame({'sample_id': self.test_ids})
            self.test_pp_x = id_df.join(self.test_pp_x, on='sample_id', how='left')

    def save_oof_df(self, preds: np.ndarray, sample_ids: np.ndarray, target_cols: List[str]):
        oof_df = pl.DataFrame(preds, schema=target_cols)
        oof_df = oof_df.with_columns(sample_id = pl.Series(sample_ids))
        oof_df.write_parquet(self.oof_path / f'oof{self.save_suffix}.parquet')

In [ ]:
# Training
save_suffix = ''
trainer = Trainer(config, model, loss_fn, save_suffix=save_suffix, logger=logger)
best_score, best_epoch = trainer.train(
    train_loader,
    valid_loader,
    colwise_best_weight=True,
    # eval_only=True,
    # eval_colwise=True,
    retrain=True,
    retrain_weight_name='model_eval239',
    retrain_best_score=0.78355,
    retrain_eval_count=239, # 前回の最終eval_countを指定する
)
logger.info(f'At Epoch={best_epoch}, Best Score={best_score:.4f}')
pred_df = trainer.test_predict(test_loader, eval_colwise=True)